# 06 - Expectations and Observables
This notebook covers the calculation of stationary and dynamic expectation values. These quantities include ensemble averaged observales measurable by experiments, correlation function and spectral densities. We recall that a (stationary) ensemble average $\langle O \rangle$ is defined by a Boltzmann-weighted average with the potential energy function $E(\cdot)$:
$$ \langle O \rangle = \mathcal{Z}^{-1}\int_{\Omega}\;\mathrm{d}x\, o(x)\exp(-\beta E(x)) $$
where $x \in \Omega$ is a molecular configuration and $o(\cdot)$ is a function which relates the atomic-coordinates of a configuration to a time-independent, microscopic observable. The function $o(\cdot)$ is often called a 'forward model', often involves computing distances or angles between particular atoms. For MSMs we have discretized our configurational space $\Omega$, which simplifies the expression to the sum

$$ \langle O \rangle = \sum_i \pi_i \mathbf{o}_i $$

where $\pi_i$ corresponding the integrated probability density of the segment of configuration space assigned to Markov state $i$. The Boltzmann distribution, or stationary distribution, $\boldsymbol{\pi}$ is computed as the left-eigenvector corresponding to the eigenvalue of 1 of the MSM transition  matrix. The mapping of the experimental observable is mapped on to the Markov states as the vector $o$ with elements,
$$ \mathbf{o}_i = \frac{1}{\pi_i\mathcal{Z}} \int_{x\in S_i} \mathrm{d}x\, o(x)\exp(-\beta E(x)) $$

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mdshare
import pyemma

## Case 1: preprocessed, two-dimensional data (toy model)

In [ ]:
file = mdshare.fetch('hmm-doublewell-2d-100k.npz', working_directory='data')
with np.load(file) as fh:
    data = fh['trajectory']

cluster = pyemma.coordinates.cluster_kmeans(data, k=50, max_iter=50)
dtrajs_all = cluster.dtrajs[0]

its = pyemma.msm.its(
    cluster.dtrajs, lags=[1, 2, 3, 5, 7, 10], nits=3, errors='bayes')

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(
    data, feature_labels=['$x$', '$y$'], ax=axes[0])
pyemma.plots.plot_density(*data.T, ax=axes[1], cbar=False, alpha=0.1)
axes[1].scatter(*cluster.clustercenters.T, s=15, c='C1')
axes[1].set_xlabel('$x$')
axes[1].set_ylabel('$y$')
pyemma.plots.plot_implied_timescales(its, ylog=False, ax=axes[2])
fig.tight_layout()

In [ ]:
msm = pyemma.msm.estimate_markov_model(cluster.dtrajs, lag=1)
bayesian_msm = pyemma.msm.bayesian_markov_model(cluster.dtrajs, lag=1)

print('fraction of states used = {:f}'.format(msm.active_state_fraction))
print('fraction of counts used = {:f}'.format(msm.active_count_fraction))

nstates = 2
pyemma.plots.plot_cktest(msm.cktest(nstates));

### An imaginary experiment
Let us first take a look at stationary ensemble averages for this system. Say the simulation data above represents a protein switching between two meta-stable configurations, perfectly separated by $y$-coordinate. Imagine then that by inspection of the two meta-stable configurations we have designed an experiment which allows us to measure an observable defined by:
$$ o(x,y) = 0.5x + y + 4. $$
We compute the observables for the entire simulation trajectory, inspect the empirical histograms and the histograms of the meta-stable sets identified by a two-state PCCA analysis.

In [ ]:
def observable(x, y): 
    return (0.5 * x + y + 4)


pcca = msm.pcca(2)
ftraj = observable(*data.T)

fig, ax = plt.subplots(figsize=(5, 4))
ax.hist(ftraj, bins=128, histtype='step', label='all data', color='k', density=True)
for num, metastable_set in enumerate(pcca.metastable_sets):
    traj_indices = np.where(np.isin(dtrajs_all, metastable_set))[0]
    ax.hist(
        ftraj[traj_indices], bins=64, density=True,
        histtype='step', label='metastable {}'.format(num + 1))
ax.legend()
ax.set_xlabel(r'$o(x, y)$')
ax.set_ylabel('empirical probability')
ax.set_xlim(-2, 7.2)
fig.tight_layout()

In the case of nonequilibrium data, one might wish to compute a reweighted instead of the empirical distribution. To do so, we have to pass the output of `msm.trajectory_weights()` as weights for the histogramming as shown in the left panel below.

Be careful not to use the discrete probabilities as weights: you would then reweight without taking the empiricial distribution of the data into account (right panel).

In [ ]:
msm_trajectory_weights = msm.trajectory_weights()[0]

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
axes[0].hist(
    ftraj, bins=128, weights=msm_trajectory_weights,
    histtype='step', label='reweighted',
    color='C0', lw=2, density=True)
axes[1].hist(
    ftraj, bins=128, weights=msm.pi[dtrajs_all],
    histtype='step', label=r'empirical $\times$ reweighted',
    color='C1', lw=2, density=True)
for ax in axes.flat:
    ax.hist(
        ftraj, bins=128,
        histtype='step', label='empirical', color='k', density=True)
    ax.set_xlabel('$o(x, y)$')
    ax.legend()
axes[0].set_ylabel('probability')
fig.tight_layout()

In the present case, the data is well equilibrated and we see no difference between the empirical and the properly reweighted distribution. The (incorrect) approach using only the MSM's stationary weights neglects the implicit distribution of the data points and, thus, yields a visibly different result.

In the experiment we have measured this observable to be equal to $3.5 \text{a.u.}$ In order to compute this experimental observable from our MSM from above, we need to compute the experimental observable for each of our Markov states by taking the average within the Markov state:
$$ \mathbf{o}_i = \frac{1}{N_{a \in S_i} } \sum_{a \in S_i} o(a_x,a_y) $$

In [ ]:
observable_vec = np.array([ftraj[dtrajs_all == i].mean()
                           for i in range(cluster.n_clusters)])

Let us visualize the observable on the cluster centers and compare to the stationary distribution of our Markov model

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharex=True, sharey=True)
pyemma.plots.plot_contour(
    *data.T,
    ftraj, ax=axes[0, 0],
    cbar_label='empirical observable',
    mask=True)
pyemma.plots.plot_density(
    *data.T, ax=axes[0, 1],
    cbar_label='empirical distribution')
pyemma.plots.plot_contour(
    *data.T,
    observable_vec[dtrajs_all],
    ax=axes[1, 0],
    cbar_label='discretized observable',
    mask=True)
pyemma.plots.plot_contour(
    *data.T,
    msm.stationary_distribution[dtrajs_all],
    ax=axes[1, 1],
    cbar_label='MSM stationary distribution',
    mask=True)
for ax in axes[1, :]:
    ax.scatter(*cluster.clustercenters.T, s=15, c='C1')
    ax.set_xlabel('$x$')
for ax in axes[:, 0]:
    ax.set_ylabel('$y$')
fig.tight_layout()

These plots illustrate how we lose some resolution when in our observable when clustering the data, since the observable is considered constant at the sample average of the all simulation frames assigned to a given cluster. With our vector of observables for our experiment we can compute the ensemble average by using the `expectation` method of our MSM:

In [ ]:
print('Experiment {:.3f} -- Prediction from MSM {:.3f}'.format(
    3.5, msm.expectation(observable_vec)))

We observe that our Markov model has a small deviation from the 'experimental' value which in a practical case we would have to gauge against imprecisions in the prediction of the observable (some contributions include: approximations involving the forward model, the dicretization/projection error and limited sampling) and the experimental uncertainty.

### Computing observables with error-bars
If we have estimated a Bayesian MSM or HMSM we can compute limited sampling contribution to our experimental observable prediction by computing sample averages and confidence intervals.

In [ ]:
observable_sample_mean = bayesian_msm.sample_mean('expectation', observable_vec)
print('Bayesian Markov state model confidence interval {:.0f}%'.format(bayesian_msm.conf * 100))

observable_ci = bayesian_msm.sample_conf('expectation', observable_vec)
print('Observable prediction: {:.2f}, {:.0f}%   CI: [{:.2f}, {:.2f}]'.format(
    observable_sample_mean, bayesian_msm.conf * 100, *observable_ci))

We have designed a new complementary experiment which is now given by

$$o_2(x,y) = (y+2)^2 + x$$

**Exercise 1**: compute the difference of our MSM prediction of this observable to the experimental value of $5\,\mathrm{a.u.}$. Compute the sample average and confidence interval from our Bayesian MSM. Is the experimental value within within the 95% confidence interval?

In [ ]:
def new_observable(x, y):
    return #FIXME


new_ftraj = #FIXME
new_observable_vec = np.array([ftraj[dtrajs_all == i].mean()
                               for i in range(cluster.n_clusters)])


def is_within(x, a, b):
    return ' not ' if x < a or x > b else ' '


experiment2 = 5.0
msm_prediction = #FIXME
bayesian_msm_sample_mean = #FIXME
bayesian_msm_ci95 = #FIXME

print('Difference between MSM and experiment {:.2f}'.format(experiment2 - msm_prediction))

print('Expl. {:.1f}, Sample mean {:.2f} and CI [{:.2f}, {:.2f}]'.format(
    experiment2,
    bayesian_msm_sample_mean,
    *bayesian_msm_ci95))
print('Experiment is{}within the 95% CI of the Bayesian MSM'.format(
    is_within(experiment2, *bayesian_msm_ci95)))

In [ ]:
def new_observable(x, y):
    return ((2 + y)**2 + x)


new_ftraj = new_observable(*data.T)
new_observable_vec = np.array([ftraj[dtrajs_all == i].mean()
                               for i in range(cluster.n_clusters)])


def is_within(x, a, b):
    return ' not ' if x < a or x > b else ' '


experiment2 = 5.0
msm_prediction = msm.expectation(new_observable_vec)

bayesian_msm_sample_mean = bayesian_msm.sample_mean('expectation', new_observable_vec)
bayesian_msm_ci95 = bayesian_msm.sample_conf('expectation', new_observable_vec)

print('Difference between MSM and experiment {:.2f}'.format(experiment2 - msm_prediction))

print('Expl. {:.1f}, Sample mean {:.2f} and CI [{:.2f}, {:.2f}]'.format(
    experiment2,
    bayesian_msm_sample_mean,
    *bayesian_msm_ci95))
print('Experiment is{}within the 95% CI of the Bayesian MSM'.format(
    is_within(experiment2, *bayesian_msm_ci95)))

## Case 2: low-dimensional molecular dynamics data (alanine dipeptide)

In [ ]:
pdb = mdshare.fetch('alanine-dipeptide-nowater.pdb', working_directory='data')
files = mdshare.fetch('alanine-dipeptide-*-250ns-nowater.dcd', working_directory='data')

feat = pyemma.coordinates.featurizer(pdb)
feat.add_backbone_torsions()
data = pyemma.coordinates.load(files, features=feat)
data_all = np.concatenate(data)

cluster = pyemma.coordinates.cluster_kmeans(
    data, k=100, max_iter=50, stride=10)
its = pyemma.msm.its(
    cluster.dtrajs, lags=[1, 2, 5, 10, 20, 50], nits=4, errors='bayes')

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(
    data_all, feature_labels=['$\Phi$', '$\Psi$'], ax=axes[0])
pyemma.plots.plot_density(*data_all.T, ax=axes[1], cbar=False, alpha=0.1)
axes[1].scatter(*cluster.clustercenters.T, s=15, c='C1')
axes[1].set_xlabel('$\Phi$')
axes[1].set_ylabel('$\Psi$')
pyemma.plots.plot_implied_timescales(its, ax=axes[2], units='ps')
fig.tight_layout()

In [ ]:
msm = pyemma.msm.estimate_markov_model(
    cluster.dtrajs, lag=10, dt_traj='0.001 ns')

print('fraction of states used = {:f}'.format(msm.active_state_fraction))
print('fraction of counts used = {:f}'.format(msm.active_count_fraction))

bayesian_msm = pyemma.msm.bayesian_markov_model(cluster.dtrajs, lag=10)

nstates = 4
pyemma.plots.plot_cktest(bayesian_msm.cktest(nstates));

## Using PyEMMA `coordinates` module to aid computation of molecular observables
We now turn to 2-Ala. In this case we want make use of features computed from molecular trajectory with the help of the the PyEMMA `coordinates` module. For this small molecular systems we will focus on the $^1H^N-^1H^{\alpha}$   $^3J$-coupling, a NMR parameter which is sensitive to the dihedral between the plane spanned by the inter-atomic vectors of H-N and N-C$\alpha$ and the plane spanned by H$\alpha$-C$\alpha$ and C$\alpha$-N. A common forward model to back-calculate $^3J$-couplings from a molecular configurations is called the Karplus equation:

$$ ^3J(\theta) = A\cos^2{\theta}+B\cos{\theta} + C, $$

Where the (Karplus) parameters $A$, $B$ and $C$ are empirical constants which depend on the properties of the atoms involved in the observable. Here we use the values $A=8.754 \, \mathrm{Hz}$, $B=−1.222\, \mathrm{Hz}$ and $C=0.111\, \mathrm{Hz}$.

In [ ]:
feat2 = pyemma.coordinates.featurizer(pdb)
feat2.add_dihedrals([[7, 6, 8, 9]])  # add relevant dihedral
print('Atoms involving dihedral: {}'.format(
    [feat2.topology.atom(i) for i in [7, 6, 8, 9]]))

dihedral = pyemma.coordinates.load(files, features=feat2)  # load to memory

def Karplus(theta):
    # define forward model
    return 8.754 * np.cos(theta)**2 - 1.222 * np.cos(theta) + 0.111


# evaluate forward model on all dihedral trajectories
ftrajs = [Karplus(t.ravel()) for t in dihedral]

# compute observable for Markov states
jcoupl_markov = np.array([
    [ft[dt == i].mean() for i in range(cluster.n_clusters)]
    for dt, ft in zip([np.hstack(cluster.dtrajs)], [np.hstack(ftrajs)])][0])

Let us visualize this observable along with the stationary distribution as above

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_contour(
    *np.concatenate(data).T,
    jcoupl_markov[np.concatenate(cluster.dtrajs)],
    ax=axes[0],
    cbar_label='$^3J$-coupling / Hz',
    mask=True)
pyemma.plots.plot_contour(
    *np.concatenate(data).T,
    msm.pi[np.concatenate(cluster.dtrajs)],
    ax=axes[1],
    cbar_label='MSM stationary distribution',
    mask=True)
for ax in axes.flat:
    ax.scatter(*cluster.clustercenters.T, s=5, c='C1')
    ax.set_xlabel('$\Phi$')
axes[0].set_ylabel('$\Psi$')
fig.tight_layout()

**Exercise 2**: predict the value of the $^3J$-coupling using the `msm` instance of 2-Ala.

In [ ]:
predicted_3j #FIXME
print('Predicted value of 3J coupling {:.3f} Hz'.format(predicted_3j))

In [ ]:
predicted_3j = msm.expectation(jcoupl_markov)
print('Predicted value of 3J coupling {:.3f} Hz'.format(predicted_3j))

**Exercise 3**: compute the sample mean and sample standard deviation of the $^3J$-coupling using the `bayesian_msm` instance of 2-Ala. (hint: use the `sample_std` method)

In [ ]:
bayesian_sample_mean_3j = #FIXME
bayesian_ci_3j = #FIXME

print('Predicted 3J coupling {:.3f} ({:.3f})'.format(bayesian_sample_mean_3j, bayesian_ci_3j))

In [ ]:
bayesian_sample_mean_3j = bayesian_msm.sample_mean('expectation', jcoupl_markov)
bayesian_ci_3j = bayesian_msm.sample_std('expectation', jcoupl_markov)

print('Predicted 3J coupling {:.3f} ({:.3f})'.format(bayesian_sample_mean_3j, bayesian_ci_3j))

## Case 3: another molecular dynamics data set (pentapeptide)

In [ ]:
pdb = mdshare.fetch('pentapeptide-impl-solv.pdb', working_directory='data')
files = mdshare.fetch('pentapeptide-*-500ns-impl-solv.xtc', working_directory='data')

feat = pyemma.coordinates.featurizer(pdb)
feat.add_backbone_torsions(cossin=True)
feat.add_sidechain_torsions(which='chi1', cossin=True)
data = pyemma.coordinates.load(files, features=feat)

tica = pyemma.coordinates.tica(data, lag=20, var_cutoff=0.9)
tica_out = np.concatenate(tica.get_output())
cluster = pyemma.coordinates.cluster_kmeans(
    tica, k=250, max_iter=50, stride=10)
its = pyemma.msm.its(cluster.dtrajs, lags=30, nits=10, errors='bayes')

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(tica_out, ax=axes[0])
pyemma.plots.plot_implied_timescales(its, ax=axes[1], dt=0.1, units='ns')
fig.tight_layout()

In [ ]:
msm = pyemma.msm.estimate_markov_model(
    cluster.dtrajs, lag=12, dt_traj='0.1 ns')

print('fraction of states used = {:f}'.format(msm.active_state_fraction))
print('fraction of counts used = {:f}'.format(msm.active_count_fraction))

bayesian_msm = pyemma.msm.bayesian_markov_model(
    cluster.dtrajs, lag=12, dt_traj='0.1 ns')

In [ ]:
nstates = 9
pyemma.plots.plot_cktest(bayesian_msm.cktest(nstates), diag=True)

hmm = msm.coarse_grain(nstates)

## Dynamic/kinetic experimental observables
Experimentally we can measure kinetic information through time-correlation functions or spectral densities. Markov models are probabilistic models of the dynamics, so if we have an ergodic dynamics described by a Markov model we can compute the equilibrium time-correlation functions using

$$ \mathbb{E}[o(x_{t})o(x_{t+k\tau})] = \underbrace{\mathbf{o}^T}_{\text{transpose of vector of observables in Markov states}} \overbrace{\boldsymbol{\Pi}}^{\text{diagonal matrix of stationary distribution}} \underbrace{\mathbf{P}^k_{\tau}}_{\text{transition matrix}} \overbrace{\mathbf{o}}^{\text{vector of observables in Markov states}} $$

we can recast this expression to

$$ \mathbb{E}[o(x_{t})o(x_{t+k\tau})] = (\mathbf{o}^T\boldsymbol{\pi})^2 + \sum^N_{i=2} \exp\left(-\frac{k\tau}{t_i}\right)(\mathbf{o}^T\boldsymbol{\phi}_i)^2 $$

by using the spectral decomposition of the transition matrix [Franks fingerprints paper]. $\phi_i$ is the $i$th left-eigenvector of $\mathbf{P}_{\tau}$, with the associated implied time-scale $t_i$. We see that the auto-correlation functions take the form of a multi-exponential decay. 

If we now consider relaxation from an non-equilibrium state $\mathbf{p_0}$, which can achieved by T-jump or P-jump experiments. In this case the initial distribution and final ensembles are not the same ($\mathbf{p_0}$ and $\boldsymbol{\pi}$) and so the auto-correlation becomes:

$$ \mathbb{E}[o(x_{t})o(x_{t+k\tau})] = (\mathbf{\hat p_0}^T\boldsymbol{\pi})(\mathbf{l}^T\boldsymbol{\pi}) + \sum^N_{i=2} \exp\left(-\frac{k\tau}{\lambda_i}\right)(\mathbf{o}^T\boldsymbol{\phi}_i)(\mathbf{\hat p_0}^T\boldsymbol{\phi}_i) $$

with $\mathbf{\hat p_0}=\boldsymbol{\Pi}^{-1}\mathbf{p_0}$.


In PyEMMA the MSM and HMSM objects have the methods `correlation` and `relaxation` which implements calculation of auto-correlation in equilibrium and relaxation from a specified non-equilibrium distribution. Many experimental observables do not measure correlation functions directly but some quantity which depends on a correlation-function, fx its Fourier transform (cite dynamical neutron scattering and olsson noe paper). In many of these cases it is possible to obtain analytical expressions of the observable which depends only on the amplitudes $(\mathbf{o}^T\boldsymbol{\phi}_i)^2$ and time-scales $t_i$, these quantities can be computed given $\mathbf{o}$ using the `fingerprint_correlation` and `fingerprint_relaxation` methods of the MSM or HMSM objects. 



In [ ]:
from mdtraj import shrake_rupley

## Trp-flourescene auto-correlation
Fluctuations in the tryptophan flourescence can be measured using spectroscopic techniques. These fluctuations depend on (among other things) on the solvent accessible surface area (SASA) of tryptophan residues. We will here use a third party library (MDTraj) to estimate the SASA using the Shrake-Rupley algorithm. Since we are not interested in inspecting the experimental observable as a function of time and since the computation can be expensive to perform for large data-sets we instead sample a representative set of configurations for each of our Markov states. Hint: A similar strategy can be used if expensive external software has to be used to compute the observables including ab initio calculations. 

In [ ]:
markov_samples = [pyemma.coordinates.save_traj(files, smpl, outfile=None, top=pdb)
                  for smpl in msm.sample_by_state(50)]

In [ ]:
# Only the first residue is Tryptophan
markov_average_trp_sasa = [shrake_rupley(sample, mode='residue')[:, 0].mean()
                           for sample in markov_samples]

Compute and plot the equilibrium auto-correlation function for Maximum Likelihood and Bayesian MSMs:

In [ ]:
eq_time_ml, eq_acf_ml = msm.correlation(markov_average_trp_sasa, maxtime=5)

eq_time_bayes, eq_acf_bayes = bayesian_msm.sample_mean(
    'correlation',
    np.array(markov_average_trp_sasa),
    maxtime=5)

eq_acf_bayes_ci_l, eq_acf_bayes_ci_u = bayesian_msm.sample_conf(
    'correlation',
    np.array(markov_average_trp_sasa),
    maxtime=5)

fig, ax = plt.subplots(figsize=(5, 4))
ax.plot(eq_time_ml, eq_acf_ml, color='orange', marker='.', label='ML MSM')
ax.plot(
    eq_time_bayes, eq_acf_bayes, ls='--',
    color='teal', label='Bayes sample mean')
ax.fill_between(
    eq_time_bayes, eq_acf_bayes_ci_l[1], eq_acf_bayes_ci_u[1],
    color='teal', alpha=0.2, lw=0)
ax.semilogx()

ax.set_xlim((eq_time_ml[1], eq_time_ml[-1]))
ax.set_xlabel(r'time / $\mathrm{ns}$')
ax.set_ylabel(r'Trp-1 SASA ACF / $\mathrm{nm}^4$')

ax.legend()
fig.tight_layout()

This signal is quite weak -- the confidence interval is approximately as large as the difference between the ACF at $\tau=0$ and at $\tau=\infty$.

We have designed an experiment that lets us initialize the ensemble in meta-stable state number 6 of `coarse_msm`.


In [ ]:
eq_time_ml, eq_acf_ml = msm.relaxation(
    hmm.metastable_distributions[6],
    markov_average_trp_sasa,
    maxtime=5)

eq_time_bayes, eq_acf_bayes = bayesian_msm.sample_mean(
    'relaxation',
    hmm.metastable_distributions[6],
    np.array(markov_average_trp_sasa),
    maxtime=5)

eq_acf_bayes_CI_l, eq_acf_bayes_CI_u = bayesian_msm.sample_conf(
    'relaxation', 
    hmm.metastable_distributions[6],
    np.array(markov_average_trp_sasa),
    maxtime=5)

fig, ax = plt.subplots(figsize=(5, 4))
ax.semilogx(eq_time_ml, eq_acf_ml, color='orange', marker='^', label='ML MSM')
ax.plot(
    eq_time_bayes, eq_acf_bayes, ls='--',
    color='teal', label='Bayes sample mean')
ax.fill_between(
    eq_time_bayes, eq_acf_bayes_CI_l[1], eq_acf_bayes_CI_u[1],
    color='teal', alpha=0.2, lw=0)
ax.semilogx()

ax.set_xlim((eq_time_ml[1], eq_time_ml[-1]))
ax.set_xlabel(r'time / $\mathrm{ns}$')
ax.set_ylabel(r'Average Trp-1 SASA / $\mathrm{nm}^2$')

ax.legend()
fig.tight_layout()

Here we see a stronger signal which is also larger than the confidence interval of the prediction.

**Exercise 4**: the plot from above however, now initialize using meta-stable state 3.

**Hint**: Copy paste the cell above and update the initial distribution at the relevant positions.

In [ ]:
#FIXME

In [ ]:
eq_time_ml, eq_acf_ml = msm.relaxation(
    hmm.metastable_distributions[3],
    markov_average_trp_sasa,
    maxtime=5)

eq_time_bayes, eq_acf_bayes = bayesian_msm.sample_mean(
    'relaxation',
    hmm.metastable_distributions[3],
    np.array(markov_average_trp_sasa),
    maxtime=5)

eq_acf_bayes_ci_l, eq_acf_bayes_ci_u = bayesian_msm.sample_conf(
    'relaxation',
    hmm.metastable_distributions[3],
    np.array(markov_average_trp_sasa),
    maxtime=5)

fig, ax = plt.subplots(figsize=(5, 4))

ax.semilogx(
    eq_time_ml, eq_acf_ml, color='orange', marker='^', label='ML MSM')
ax.plot(
    eq_time_bayes, eq_acf_bayes, ls='--',
    color='teal', label='Bayes sample mean')
ax.fill_between(
    eq_time_bayes, eq_acf_bayes_ci_l[1], eq_acf_bayes_ci_u[1],
    color='teal', alpha=0.2, lw=0)
ax.semilogx()

ax.set_xlim((eq_time_ml[1], eq_time_ml[-1]))
ax.set_xlabel(r'time / $\mathrm{ns}$')
ax.set_ylabel(r'Average Trp-1 SASA / $\mathrm{nm}^2$')

ax.legend()
fig.tight_layout()

A new experiment allows us to measure the quantity:

$$ \sum_{i=2}^N \overbrace{t_i}^{\text{implied timescale }i} \underbrace{(\mathbf{o}^T\boldsymbol{\phi}_i)}_{\text{Fingerprint amplitude } i}$$

For where $\mathbf{o}$ is the Trp-1 SASA mapped onto the Markov states. 

**Exercise 5**: using the `fingerprint_correlation` method compute the timescales and amplitudes of this new observable.

In [ ]:
def new_observable(timescales, amplitudes):
    return timescales[1:].dot(amplitudes[1:])


timescales, amplitudes = #FIXME

print('Value of new observable {:.3e} nm^2 ns'.format(
    new_observable(timescales, amplitudes)))

In [ ]:
def new_observable(timescales, amplitudes):
    return timescales[1:].dot(amplitudes[1:])


timescales, amplitudes = msm.fingerprint_correlation(markov_average_trp_sasa)

print('Value of new observable {:.3e} nm^2 ns'.format(
    new_observable(timescales, amplitudes)))

## Wrapping up
In this notebook we have covered how to compute ensemble averaged properties/observables from Markov state models and hidden Markov state models. We have further covered the quantification of uncertainty in these predictions. We have specifically discussed the MSM/HMSM object methods
* `expectation()` computes the stationary ensemble average of a property.
* `correlation()` computes the equilibrium auto/cross-correlation function of a/two property/properties.
* `relaxation()` computes the relaxation auto/cross-correlation function of a/two property/properties given an initial state distribution.
* `fingerprint_correlation()` computes the amplitudes and time-scales of the MSM/HMSM auto/cross-correlation function.
* `fingerprint_relaxation()` computes the amplitudes and time-scales of the MSM/HMSM relaxation auto/cross-correlation function.
We have further covered the following object methods to compute statistics from Bayesian MSM/HMSMs:
* `sample_mean()` computes the mean of a property over the sampled MSMs/HMSMs in a Bayesian model. 
* `sample_conf()` computes the confidence interval of a property over the sampled MSMs/HMSMs in a Bayesian model.
* `sample_std()` computes the standard deviation of a property over the sampled MSMs/HMSMs in a Bayesian model.

Finally, we have shown how to use these methods together with precomputed observables or observables computed using third-party libraries. 